In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.1s remaining:    3.6s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.1s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


In [3]:
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=0.01, target_snr=25)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
 50%|█████     | 1/2 [00:00<00:00,  2.45it/s]c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
100%|██████████| 2/2 [00:00<?, ?it/s]


source data shape:  (1284, 10) (1284, 10)


100%|██████████| 2/2 [00:00<00:00, 664.34it/s]


Using pyvistaqt 3d backend.

Using control points [1.22247783e-08 2.38480528e-08 6.58575229e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


In [4]:
from invert import Solver
from invert.config import all_solvers
from invert.evaluate import nmse, corr
from invert.adapters import contextualize_bd
import pickle as pkl

if "LUCAS" in all_solvers:
    all_solvers.remove("LUCAS")

settings = dict(number_of_sources=(1,10), extents=(1, 40), duration_of_trial=1, target_snr=(1,25))
errors = {sname: [] for sname in all_solvers}
solvers = dict()

for i in range(200):
    # print(i)
    sim = Simulation(fwd, info, settings).simulate(2)
    stc = sim.source_data[0]
    evoked = sim.eeg_data[0].average()

    for solver_name in all_solvers:
        print(solver_name)
        solver = Solver(solver=solver_name)
        if (not solver_name in solvers) or ("sparse" in solver_name.lower() or "bayes" in solver_name.lower()):
            solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha="auto")
        stc_hat = solvers[solver_name].apply_inverse_operator(evoked)
        # stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))
        error = np.mean(corr(stc.data, stc_hat.data))
        errors[solver_name].append( error )
        
        solver_name = "c" + solver_name
        if not solver_name in errors:
            errors[solver_name] = []
        stc_hat = contextualize_bd(stc_hat, fwd, fast=True)
        error = np.mean(corr(stc.data, stc_hat.data))
        
        errors[solver_name].append( error )



    if i > 22:
        fn = "errors.pkl"
        with open(fn, 'wb') as f:
            pkl.dump(errors, f)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 400.01it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 12.99it/s]


MNE
wMNE
dSPM
alpha must be set to a float when using Dynamic Statistical Parametric Mapping, auto does not work yet.
LORETA
sLORETA
eLORETA


In [7]:
import pickle as pkl
fn = "errors.pkl"
with open(fn, 'rb') as f:
    errors = pkl.load(f)

In [8]:
import pandas as pd
import seaborn as sns
sns.set(font_scale=0.8)
df = pd.DataFrame(errors)
sorted_index = df.median().sort_values().index
df = df[sorted_index]

plt.figure()
sns.boxplot(data=df)
plt.title("Correlation with ground truth")


df_mean_var = pd.concat([df.mean(), df.std()], axis=1)
df_mean_var = df_mean_var.rename(columns={0: "Median", 1: "Variance"})
df_mean_var["MedVar"] = df_mean_var["Median"] / df_mean_var["Variance"]
df_mean_var["Method"] = df_mean_var.index
display(df_mean_var)

plt.figure()
sns.scatterplot(x="Median", y="Variance", hue="Method", size="MedVar", data=df_mean_var)
plt.xlabel("Median")
plt.ylabel("Variance")


,Median,Variance,MedVar,Method
cBayesian Beamformer,0.002265,0.027258,0.083097,cBayesian Beamformer
Bayesian Beamformer,0.002182,0.053111,0.041083,Bayesian Beamformer
cBackus-Gilbert,0.044199,0.115713,0.381973,cBackus-Gilbert
Backus-Gilbert,0.035868,0.094927,0.377851,Backus-Gilbert
cBayesian LORETA,0.077309,0.082330,0.939014,cBayesian LORETA
Bayesian LORETA,0.072263,0.065586,1.101808,Bayesian LORETA
wMNE,0.080970,0.027878,2.904499,wMNE
cwMNE,0.096279,0.039392,2.444103,cwMNE
cLAURA,0.127542,0.118207,1.078978,cLAURA
cMultiple Sparse Priors,0.102152,0.067876,1.504986,cMultiple Sparse Priors


Text(0, 0.5, 'Variance')